# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [50]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
# Import custom methods to work with Apache Cassandra
from lib import generate_str, create_table, insert_query

#### Creating list of filepaths to process original event csv data files

In [51]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [52]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            # The rows without artist information (empty strings) are skipped
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [53]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [54]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [55]:
try:
    query = "CREATE  KEYSPACE IF NOT EXISTS sparkify \
             WITH REPLICATION = {'class' : 'SimpleStrategy',\
                                 'replication_factor' : 1}"
    session.execute(query)

except Exception as e:
    print(e)

#### Set Keyspace

In [56]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run. As a best practice it is recomended to run assign one table per query since JOIN and GROUP BY statements are not supported in Apache Cassandra.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

CQL Query:
```
SELECT artist, song, length FROM <table_name> WHERE sessionId = 228 AND itemInSession = 4
```

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

CQL Query: song must be sorted by itemInSession
```
SELECT artist, song, firstName, lastName FROM <table_name> WHERE userid = 10 AND sessionid = 182
```

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

CQL Query:
```
SELECT firstName, lastName FROM <table_name> WHERE song = 'All Hands Against His Own'
```

In [57]:
# For further insight we are going to load all the data in './event_datafile_new.csv' 
# into a single table with sessionId as PRIMARY KEY. 
# This is thought as a helper table just in case we want to take a look into certain fields.

# "table_events" creation
table = "table_events"
fields_events = "(sessionId int, \
            artist text, \
            firstName text, \
            gender text, \
            itemInSession int, \
            lastName text, \
            length double, \
            level text, \
            location text, \
            song text, \
            userId int, \
            PRIMARY KEY (sessionId))"
create_table(table_name = table, fields = fields_events, session = session)

# data insertion into "table_events"
file = 'event_datafile_new.csv'
fields = ['sessionId','artist','firstName','gender','itemInSession','lastName','length',\
          'level','location','song','userId']

with open(file, encoding = 'utf8') as f:
    print("Inserting ./"+file+" data into table "+table)
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    
    for line in csvreader:
        query = insert_query(fields, table)
        # Since we are reading from CSV files we need to transform the type of the fields to
        # match the schema of the table. There is the option to implement this automatically
        # with the QUOTE_NONNUMERIC option [1] but this way we have more control over the data
        #
        # [1] https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        values = [int(line[8]),line[0], line[1], line[2], int(line[3]), line[4], float(line[5]),\
          line[6], line[7], line[9], int(line[10])]

        session.execute(query, values)

Table table_events was dropped
Table table_events was created
Inserting ./event_datafile_new.csv data into table table_events


In [58]:
# To verify that all the rows were inserted we will run two queries:

# 1 - row count: "SELECT COUNT(*) FROM table1"
print("Query: " + "SELECT COUNT(*) FROM {}".format(table))
print("Output:")
rows = session.execute("SELECT COUNT(*) FROM {}".format(table))
for r in rows:
    print(r)

print("\nQuery: " + "SELECT * FROM {} LIMIT 5".format(table))
print("Output:")
# 2 - check all the data: "SELECT * FROM table1"
rows = session.execute("SELECT * FROM {} LIMIT 5".format(table))
for r in rows:
    print(r[:])

Query: SELECT COUNT(*) FROM table_events
Output:
Row(count=776)

Query: SELECT * FROM table_events LIMIT 5
Output:
(23, 'Max Richter', 'Layla', 'F', 33, 'Griffin', 193.38404, 'paid', 'Lake Havasu City-Kingman, AZ', 'Organum', 24)
(660, 'Mariah Carey', 'Ryan', 'M', 2, 'Smith', 261.82485, 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'Bye Bye', 26)
(893, 'After 7', 'Tegan', 'F', 21, 'Levine', 241.94567, 'paid', 'Portland-South Portland, ME', 'Nights Like This', 80)
(53, 'Bloodhound Gang', 'Kaleb', 'M', 0, 'Cook', 260.20526, 'free', 'Yuba City, CA', 'Uhn Tiss Uhn Tiss Uhn Tiss', 54)
(987, 'Black Eyed Peas', 'Chloe', 'F', 18, 'Cuevas', 229.61587, 'paid', 'San Francisco-Oakland-Hayward, CA', "Let's Get It Started", 49)


### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

CQL Query:
```
SELECT artist, song, length FROM <table_name> WHERE sessionId = 338 AND itemInSession = 4
```

In [59]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

table = "artist_song"
fields1_create = "(sessionId int,\
            itemInSession int, \
            artist text, \
            length double, \
            song text, \
            PRIMARY KEY (sessionId, itemInSession))"

create_table(table_name = table, fields = fields1_create, session = session)

Table artist_song was dropped
Table artist_song was created


In [60]:
file = 'event_datafile_new.csv'
fields1 = ['sessionId','itemInSession','artist','length','song']

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    
    for line in csvreader:
        query = insert_query(fields1, table)
        # Since we are reading from CSV files we need to transform the type of the fields to
        # match the schema of the table. There is the option to implement this automatically
        # with the QUOTE_NONNUMERIC option [1] but this way we have more control over the data
        #
        # [1] https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        values = [int(line[8]),int(line[3]),line[0],float(line[5]),line[9]]
        
        session.execute(query, values)

In [61]:
# We now run the query1 to verify that the table is modeled according to the desired output

query1 = "SELECT artist, song, length FROM {} WHERE sessionId = 338 AND itemInSession = 4".format(table)

rows = session.execute(query1)
for r in rows:
    #print(r.artist, r.song, r.length)
    print(r[:])
    
# Uncomment this part to check the total number of rows in table1    
# query_count_check = "SELECT COUNT(*) FROM {}".format(table)

#rows = session.execute(query_count_check)
#for r in rows:
#    print(r)

('Faithless', 'Music Matters (Mark Knight Dub)', 495.3073)


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

CQL Query: song must be sorted by itemInSession
```
SELECT artist, song, firstName, lastName FROM <table_name> WHERE userid = 10 AND sessionid = 182
```

In [62]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

table = "user_playlist_songs"
fields2_create = "(userId int, \
            sessionId int, \
            itemInSession int, \
            artist text, \
            song text, \
            firstName text, \
            lastName text, \
            PRIMARY KEY ((userid, sessionid), itemInSession))"

create_table(table_name = table, fields = fields2_create, session = session)                    

Table user_playlist_songs was dropped
Table user_playlist_songs was created


In [63]:
file = 'event_datafile_new.csv'
fields2 = ['userId','sessionId','itemInSession','artist','song','firstName','lastName']

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    
    for line in csvreader:
        query = insert_query(fields2, table)
        # Since we are reading from CSV files we need to transform the type of the fields to
        # match the schema of the table. There is the option to implement this automatically
        # with the QUOTE_NONNUMERIC option [1] but this way we have more control over the data
        #
        # [1] https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        values = [int(line[10]),int(line[8]),int(line[3]),line[0], line[9], line[1], line[4]]
        
        session.execute(query, values)

In [64]:
# We now run the query2 to verify that the table is modeled according to the desired output

query2 = "SELECT iteminsession, artist, song, firstName, lastName FROM {} WHERE userid = 10 AND sessionid = 182".format(table)

rows = session.execute(query2)
for r in rows:
    #print(r.iteminsession, r.artist, r.song, r.firstname, r.lastname)
    print(r[:])

# Uncomment this part to check the total number of rows in table2    
#query_count_check = "SELECT COUNT(*) FROM {}".format(table)

#rows = session.execute(query_count_check)
#for r in rows:
#    print(r)

(0, 'Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz')
(1, 'Three Drives', 'Greece 2000', 'Sylvie', 'Cruz')
(2, 'Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz')
(3, 'Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

CQL Query:
```
SELECT firstName, lastName FROM <table_name> WHERE song = 'All Hands Against His Own'
```

In [65]:
# Query 3: Give me every user name (first and last) 
# in my music app history who listened to the song 'All Hands Against His Own'

table = "user_song"
fields2_create = "(song text, \
            userId int, \
            firstName text, \
            lastName text, \
            PRIMARY KEY (song, userId))"

create_table(table_name = table, fields = fields2_create, session = session)

Table user_song was dropped
Table user_song was created


In [66]:
file = 'event_datafile_new.csv'
fields3 = ['song','userId','firstName','lastName']

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    
    for line in csvreader:
        query = insert_query(fields3, table)
        # Since we are reading from CSV files we need to transform the type of the fields to
        # match the schema of the table. There is the option to implement this automatically
        # with the QUOTE_NONNUMERIC option [1] but this way we have more control over the data
        #
        # [1] https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        values = [line[9], int(line[10]), line[1], line[4]]        
        
        session.execute(query, values)

In [67]:
# We now run the query3 to verify that the table is modeled according to the desired output

query3 = "SELECT firstName, lastName FROM {} WHERE song = 'All Hands Against His Own'".format(table)

rows = session.execute(query3)
for r in rows:
    print(r.firstname, r.lastname)

# Uncomment this part to check the total number of rows in table3    
#query_count_check = "SELECT COUNT(*) FROM {}".format(table)

#rows = session.execute(query_count_check)
#for r in rows:
#    print(r)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [68]:
# Clean up: the tables are dropped before closing out the session
for table in ["table_events", "artist_song", "user_playlist_songs", "user_songs"]:
    session.execute("DROP TABLE IF EXISTS {}".format(table))
    print("Table {} was dropped".format(table))

Table table_events was dropped
Table artist_song was dropped
Table user_playlist_songs was dropped
Table user_songs was dropped


### Close the session and cluster connection¶

In [69]:
session.shutdown()
cluster.shutdown()